In [ ]:
import numpy as np
import collections
import matplotlib.pyplot as plt
import tensorflow as tf
%tensorflow_version 1.4
import pandas as pd
%matplotlib inline

In [ ]:
print(tf.__version__)

In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1kamomgHvR4iSlfCfUoCmb32v0KBGuGyB"})
downloaded.GetContentFile('segmented_train_seg_by_word.txt')  

In [ ]:
tf.test.gpu_device_name()

In [ ]:
!/opt/bin/nvidia-smi

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
enline = None
chline = None

sentlength = 10

enlines = []
chlines = []
num = 0

with open("/content/segmented_train_seg_by_word.txt") as hfd1:
  for line in hfd1:
    num += 1
    if num % 2 == 1:
      enline = line
      continue
    else:
      chline = line
    
    enlinesp = [i.lower() for i in enline.strip("\n").split()]
    chlinesp = [i for i in chline.strip("\n").split()]
    
    if len(enlinesp) <= sentlength and len(chlinesp) <= sentlength:
      enlines.append(enlinesp)
      chlines.append(chlinesp)

In [ ]:
len(enlines), len(chlines)

In [ ]:
enlines[:10]

In [ ]:
chlines[:10]

In [ ]:
enwords = []
chwords = []
for sent in chlines:
  for word in sent:
    chwords.append(word)

for sent in enlines:
  for word in sent:
    enwords.append(word)

In [ ]:
from collections import Counter

In [ ]:
Counter(chwords).most_common(50)

In [ ]:
Counter(chwords).most_common()[-10:-1]

In [ ]:
len(Counter(chwords).most_common()), len(Counter(enwords).most_common())

In [ ]:
# put new words or placeholder into dictionary
def addwords(x2ind, ind2x, w):
  if w in x2ind:
    return
  ind2x[len(x2ind)] = w
  x2ind[w] = len(ind2x)-1 # important

In [ ]:
ch2ind = {}
ind2ch = {}
en2ind = {}
ind2en = {}

specialchars = ['<end>', '<unk>', '<eos>', '<start>']

for char in specialchars:
  addwords(ch2ind, ind2ch, char)
  addwords(en2ind, ind2en, char)

In [ ]:
ch2ind

In [ ]:
ind2ch

In [ ]:
n = 0
m =0
for word,_ in Counter(chwords).most_common(150000):
  n += 1
  addwords(ch2ind, ind2ch, word)

for word,_ in Counter(enwords).most_common(150000):
  m += 1
  addwords(en2ind, ind2en, word)

In [ ]:
n,m

In [ ]:
en2ind['<end>'], ind2en[en2ind['<end>']]

In [ ]:
encoder_x_in = []
decoder_y_in = []
decoder_y_out = []

input_x_len = []
input_y_len = []

for chsent in chlines:
  indsent = [ch2ind.get(i, ch2ind['<unk>']) for i in chsent]
  decoder_y_in.append([ch2ind['<start>']] + indsent)
  decoder_y_out.append(indsent + [ch2ind['<end>']])
  input_y_len.append(len(indsent) + 1)

for ensent in enlines:
  indsent = [en2ind.get(i, en2ind['<unk>']) for i in ensent]
  indsent.append(en2ind['<eos>'])
  encoder_x_in.append(indsent)
  input_x_len.append(len(indsent))# !!!

In [ ]:
[ind2en[i] for i in encoder_x_in[1020]]

In [ ]:
[ind2ch[i] for i in decoder_y_in[1020]]

In [ ]:
[ind2ch[i] for i in decoder_y_out[1020]]

In [ ]:
input_x_len[1020], input_y_len[1020]

In [ ]:
len(encoder_x_in),\
len(decoder_y_in),\
len(decoder_y_out),\
len(input_x_len),\
len(input_y_len)

### Modeling

In [ ]:
from tensorflow.python.layers import core as layers_core

In [ ]:
import tflearn
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True,allow_soft_placement = True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)

In [ ]:
embedding_size = 512
num_units = 512
batch_size = 128
layer_number = 2
max_grad = 1.0
dropout = 0.2
src_vocab_size = len(en2ind)
target_vocab_size = len(ch2ind)
seq_max_len = sentlength + 1
maximum_iterations = 10

In [ ]:
tf.reset_default_graph()
config = tf.ConfigProto(log_device_placement=True,allow_soft_placement = True)
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.4
session = tf.Session(config=config)

with tf.device('/device:GPU:0'):
    initializer = tf.random_uniform_initializer(
        -0.08, 0.08)
    tf.get_variable_scope().set_initializer(initializer)
    
    x = tf.placeholder("int32", [None, None])
    y = tf.placeholder("int32", [None, None])
    y_in = tf.placeholder("int32",[None,None])
    
    x_len = tf.placeholder("int32",[None])
    y_len = tf.placeholder("int32",[None])
    learning_rate = tf.placeholder(tf.float32, shape=[])
    
    # embedding
    embedding_encoder = tf.get_variable(
        "embedding_encoder", [src_vocab_size, embedding_size],dtype=tf.float32)
    embedding_decoder = tf.get_variable(
        "embedding_decoder", [target_vocab_size, embedding_size],dtype=tf.float32)
    
    encoder_emb_inp = tf.nn.embedding_lookup(
        embedding_encoder, x)
    decoder_emb_inp = tf.nn.embedding_lookup(
        embedding_decoder, y_in)
    
    # encoder
    num_bi_layers = int(layer_number / 2)
    
    # Build RNN cell
    encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

    # Run Dynamic RNN
    #   encoder_outputs: [max_time, batch_size, num_units]
    #   encoder_state: [batch_size, num_units]
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(
        encoder_cell, encoder_emb_inp,
        sequence_length=x_len, time_major=False, dtype=tf.float32)
        
    
    
    batch_size_in = tf.shape(x)[0]
    projection_layer = layers_core.Dense(
        len(ch2ind), use_bias=False)
    # Dynamic decoding
    with tf.variable_scope("decode_layer"):
        # Build RNN cell
        decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)
        # Helper
        helper = tf.contrib.seq2seq.TrainingHelper(
            decoder_emb_inp, y_len, time_major=False)
        # Decoder
        decoder = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper, encoder_state,
            output_layer=projection_layer)
        # Dynamic decoding
        outputs, _ , __ = tf.contrib.seq2seq.dynamic_decode(decoder)
        logits = outputs.rnn_output
        target_weights = tf.sequence_mask(
            y_len, seq_max_len, dtype=logits.dtype)
    
    # predicting
    # Helper
    with tf.variable_scope("decode_layer"):
        # Helper
        helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
            embedding_decoder,
            tf.fill([batch_size_in], ch2ind['<start>']), ch2ind['<end>'])

        # Decoder
        decoder = tf.contrib.seq2seq.BasicDecoder(
            decoder_cell, helper, encoder_state,
            output_layer=projection_layer)
        # Dynamic decoding
        outputs, _ , __= tf.contrib.seq2seq.dynamic_decode(
            decoder, maximum_iterations=maximum_iterations)
        translations = outputs.sample_id
        

    # calculate loss
    crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=y, logits=logits)
    train_loss = (tf.reduce_sum(crossent * target_weights) /
        tf.cast(batch_size_in,tf.float32))
    
    optimizer_ori = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    trainable_params = tf.trainable_variables()
    gradients = tf.gradients(train_loss, trainable_params)
    clip_gradients, _ = tf.clip_by_global_norm(gradients, max_grad)
    global_step = tf.Variable(0, trainable=False, name='global_step')
    optimizer = optimizer_ori.apply_gradients(
            zip(clip_gradients, trainable_params), global_step=global_step)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(train_loss)
    #trainop = tflearn.TrainOp(loss=train_loss, optimizer=optimizer,
    #                          metric=train_loss, batch_size=64)

 

In [ ]:
session.run(tf.global_variables_initializer())

In [ ]:
encoder_x_in[:1]

In [ ]:
session.run(encoder_emb_inp,feed_dict={
    x:np.asarray(encoder_x_in[:1])
}).shape

In [ ]:
encoder_x_in = tf.keras.preprocessing.sequence.pad_sequences(encoder_x_in,padding='post',value=en2ind['<eos>'])
decoder_y_in = tf.keras.preprocessing.sequence.pad_sequences(decoder_y_in,padding='post',value=en2ind['<end>'])
decoder_y_out = tf.keras.preprocessing.sequence.pad_sequences(decoder_y_out,padding='post',value=en2ind['<end>'])

input_x_len = np.asarray(input_x_len)
input_y_len = np.asarray(input_y_len)

In [ ]:
encoder_x_in.shape,decoder_y_in.shape,decoder_y_out.shape,input_x_len.shape,input_y_len.shape

In [ ]:
print("start modeling……")
losses = []
beginning_lr = 4
for one_epoch in range(0,20):
    index = np.asarray(list(range(len(encoder_x_in))))
    np.random.shuffle(index)
    for i in range(0, 20000, batch_size):
        batchindex = index[i:i + batch_size]
        
        batch_lr = beginning_lr if one_epoch < 20 else beginning_lr * 0.5 ** (one_epoch - 20)
        if len(batchindex) < batch_size:
            break
        _,batch_loss = session.run([optimizer,train_loss],feed_dict={
            x:encoder_x_in[batchindex],
            y:decoder_y_out[batchindex],
            y_in:decoder_y_in[batchindex],

            x_len:input_x_len[batchindex],
            y_len:input_y_len[batchindex],
            learning_rate:batch_lr,
        })

        print(batch_loss, batch_lr, i)
        losses.append(batch_loss)

In [ ]:
pd.DataFrame(losses).plot()

In [ ]:
def translate(sent):
    senttoken = [en2ind[i.lower()] for i in sent.split()]
    senttoken.append(en2ind['<eos>'])
    inputx = np.asarray([senttoken])
    inputx_len = np.asarray([len(senttoken)])
    print(inputx,inputx_len)
    batch_translations = session.run(translations,feed_dict={
            x:inputx,
            x_len:inputx_len,
        })[0]
    return ''.join([ind2ch[i] for i in batch_translations])

In [ ]:
source_sentence = "I love english ."

In [ ]:
translate(source_sentence)